<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# Lab: Métricas, Regressão Logística & K-vizinhos mais Próximos

Referências e Materiais úteis para este Lab: *ver notas de aula de regressão logística e K-vizinhos mais Próximos*. Acesse também https://en.wikipedia.org/wiki/Precision_and_recall.

<br>

---





# Caso: **Predição de Diagnósticos a partir de Dados de Imagens**


https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

Os dados acima são características computadas a partir de uma imagem digitalizada por *agulha fina* (PAAF) de uma massa mamária. Eles descrevem características dos núcleos celulares presentes na imagem e classificam os tumores como malignos ou benignos.



<img src="https://github.com/Rogerio-mack/IMT_Ciencia_de_Dados/blob/main/images/ClassificationBreastCancer.PNG?raw=true" width=1200, align="right">

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
#@markdown **Set Up Lab** (*run this before continue*)
breast = pd.read_csv('https://github.com/Rogerio-mack/Machine-Learning-I/raw/main/data/breast.csv')

from sklearn.model_selection import train_test_split

seed = 1984
X_train, X_test, y_train, y_test = train_test_split(breast.drop(columns=['diagnosis']), breast.diagnosis, test_size=0.20, stratify=breast.diagnosis, random_state=seed)

breast = pd.concat([X_train, y_train],axis=1)
new_breast = X_test

breast.to_csv('breast.csv',index=False)

new_breast['diagnosis'] = '?'
new_breast.to_csv('new_breast.csv',index=False)



E você empregará os dados já tratados no set up acima, com os dataframes `breast` e `new_breast`. A idea aqui é empregar os dados de `breast` para estimar as classes dos novos casos desconhecidos de `new_breast`.



In [3]:
breast.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
106,91505,12.540,16.32,81.25,476.3,0.1158,0.10850,0.05928,0.03279,0.1943,...,21.40,86.67,552.0,0.1580,0.1751,0.1889,0.08411,0.3155,0.07538,B
500,90524101,17.990,20.66,117.80,991.7,0.1036,0.13040,0.12010,0.08824,0.1992,...,25.41,138.10,1349.0,0.1482,0.3735,0.3301,0.19740,0.3060,0.08503,M
68,915186,9.268,12.87,61.49,248.7,0.1634,0.22390,0.09730,0.05252,0.2378,...,16.38,69.05,300.2,0.1902,0.3441,0.2099,0.10250,0.3038,0.12520,B
515,897880,10.050,17.53,64.41,310.8,0.1007,0.07326,0.02511,0.01775,0.1890,...,26.84,71.98,384.0,0.1402,0.1402,0.1055,0.06499,0.2894,0.07664,B
138,881046502,20.580,22.14,134.70,1290.0,0.0909,0.13480,0.16400,0.09561,0.1765,...,27.84,158.30,1656.0,0.1178,0.2920,0.3861,0.19200,0.2909,0.05865,M


In [4]:
new_breast.head()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
215,904689,12.96,18.29,84.18,525.2,0.07351,0.07899,0.04057,0.01883,0.1874,...,24.61,96.31,621.9,0.09329,0.23180,0.1604,0.06608,0.3207,0.07247,?
39,902975,12.21,14.09,78.78,462.0,0.08108,0.07823,0.06839,0.02534,0.1646,...,19.29,87.65,529.9,0.10260,0.24310,0.3076,0.09140,0.2677,0.08824,?
99,871201,19.59,18.15,130.70,1214.0,0.11200,0.16660,0.25080,0.12860,0.2027,...,26.39,174.90,2232.0,0.14380,0.38460,0.6810,0.22470,0.3643,0.09223,?
70,846226,19.17,24.80,132.40,1123.0,0.09740,0.24580,0.20650,0.11180,0.2397,...,29.94,151.70,1332.0,0.10370,0.39030,0.3639,0.17670,0.3176,0.10230,?
264,90769601,11.13,16.62,70.47,381.1,0.08151,0.03834,0.01369,0.01370,0.1511,...,20.29,74.35,421.1,0.10300,0.06219,0.0458,0.04044,0.2383,0.07083,?


# Exercício

Empregue a regressão logística (`max_iter=10000`) para classificar os casos como malignos ou benignos.

* Empregue os seguintes parâmetros para separação dos conjuntos de treinamento e teste `test_size=0.3, stratify=y, random_state=1`.
* Observe, ao menos um atributo não pode ser empregado no treinamento.
* Faça a normalização dos dados (por simplicidade, sendo todos valores positivos, divida o valor de cada atributo pelo maior da coluna).
* Não empregue outros parâmetros não informados aqui pois os resultados podem divergir.

Analise o desempenho do modelo obtido e em seguida empregue para determinar o diagnóstico provável dos novos casos (`new_breast`).

(\*) **Atenção:** *Na predição, não esqueça de aplicar a *mesma* transformação (a normalização) empregada durante o treinamento!*

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Entradas e Saídas
X = breast.drop(columns=['id','diagnosis'])
X = X / X.max()
y = breast['diagnosis']

# Separando conjuntos de Treinamento e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1)

# Definição do modelo
clf = LogisticRegression(max_iter=10000)

# Treinamento
clf.fit(X_train,y_train)


LogisticRegression(max_iter=10000)

In [6]:
# Avaliação
y_pred = clf.predict(X_test)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

cm = confusion_matrix(y_test, y_pred)
print('\nMatriz de Confusão:\n')
print(cm)
print()

cm_df = pd.DataFrame(cm,index=[clf.classes_],columns=[clf.classes_])
display(cm_df)
print()

accuracy = accuracy_score(y_test, y_pred)
print('\nScore de Acuracidade (1):\n')
print(f'{accuracy:.4f}')

accuracy = clf.score(X_test, y_test)
print('\nScore de Acuracidade (2):\n')
print(f'{accuracy:.4f}')

print('\nClassification Report:\n')
print(classification_report(y_test, y_pred))


Matriz de Confusão:

[[82  0]
 [ 3 45]]



,B,M
B,82,0
M,3,45




Score de Acuracidade (1):

0.9769

Score de Acuracidade (2):

0.9769

Classification Report:

              precision    recall  f1-score   support

           B       0.96      1.00      0.98        82
           M       1.00      0.94      0.97        48

    accuracy                           0.98       130
   macro avg       0.98      0.97      0.97       130
weighted avg       0.98      0.98      0.98       130



Q1. Qual a acuracidade geral do modelo?

0.9769

Q2. Qual a precisão obtida para classe M?

1.00

Q3. Qual classe teve Falso Positivos? Qual o percentual de Falso Positivos para essa classe?

B, 0.04

Q4. Qual classe não teve todos as instâncias identificadas corretamente? Qual o percentual de Falso Negativos para essa classe?

M, 0.06

Q5. Quantos casos benignos e malignos classificados nos novos casos?

71 e 37




$$ Precisão = \frac{TP}{TP +FP}$$

In [7]:
print(f'Precisão de B = {cm_df.loc["B"]["B"].values[0][0]} / ({cm_df.loc["B"]["B"].values[0][0]} + {cm_df.loc["M"]["B"].values[0][0]}) = {cm_df.loc["B"]["B"].values[0][0] / (cm_df.loc["B"]["B"].values[0][0] + cm_df.loc["M"]["B"].values[0][0]):.2f}')

Precisão de B = 82 / (82 + 3) = 0.96


In [8]:
print(f'Precisão de M = {cm_df.loc["M"]["M"].values[0][0]} / ({cm_df.loc["M"]["M"].values[0][0]} + {cm_df.loc["B"]["M"].values[0][0]}) = {cm_df.loc["M"]["M"].values[0][0] / (cm_df.loc["M"]["M"].values[0][0] + cm_df.loc["B"]["M"].values[0][0]):.2f}')

Precisão de M = 45 / (45 + 0) = 1.00


$$ Recall = \frac{TP}{TP + FN}$$

In [9]:
print(f'Recall de B = {cm_df.loc["B"]["B"].values[0][0]} / ({cm_df.loc["B"]["B"].values[0][0]} + {cm_df.loc["B"]["M"].values[0][0]}) = {cm_df.loc["B"]["B"].values[0][0] / (cm_df.loc["B"]["B"].values[0][0] + cm_df.loc["B"]["M"].values[0][0]):.2f}')

Recall de B = 82 / (82 + 0) = 1.00


In [10]:
print(f'Recall de M = {cm_df.loc["M"]["M"].values[0][0]} / ({cm_df.loc["M"]["M"].values[0][0]} + {cm_df.loc["M"]["B"].values[0][0]}) = {cm_df.loc["M"]["M"].values[0][0] / (cm_df.loc["M"]["M"].values[0][0] + cm_df.loc["M"]["B"].values[0][0]):.2f}')

Recall de M = 45 / (45 + 3) = 0.94


$$ F1 = \frac{2}{\frac{1}{Precisão} + \frac{1}{Recall}} $$

In [11]:
print(f'F1 de B = 2 / (1/0.96 + 1/1) = { 2 / (1/0.96 + 1/1) :.2f}')

F1 de B = 2 / (1/0.96 + 1/1) = 0.98


In [12]:
print(f'F1 de M = 2 / (1/1 + 1/0.94) = { 2 / (1/1 + 1/0.94) :.2f}')

F1 de M = 2 / (1/1 + 1/0.94) = 0.97


Atenção, aplique a mesma transformação empregada sobre o primeiro conjunto!

In [13]:
# Novos casos
X_new = new_breast.drop(columns=['id','diagnosis'])
X_new = X_new / breast.drop(columns=['id','diagnosis']).max()
y_pred = clf.predict(X_new)

new_breast.diagnosis = y_pred

new_breast.diagnosis.value_counts()

B    71
M    37
Name: diagnosis, dtype: int64

# Exercício

Empregue agora o modelo de K-vizinhos mais próximos, para k=3 e k=5 e analise o desempenho de cada modelo. Em seguida empregue o melhor modelo Knn para determinar o diagnóstico provável dos novos casos.

* Empregue os mesmos dados de treinamento e teste empregados antes.

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# Entradas e Saídas
# X = breast.drop(columns=['id','diagnosis'])
# X = X / X.max()
# y = breast['diagnosis']

# Separando conjuntos de Treinamento e Teste
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1)

# Definição do modelo
clf = KNeighborsClassifier(n_neighbors = 5)

# Treinamento
clf.fit(X_train,y_train)

# Avaliação
y_pred = clf.predict(X_test)

print(clf)
print()

cm = confusion_matrix(y_test, y_pred)
print('\nMatriz de Confusão:\n')
print(cm)
print()

cm_df = pd.DataFrame(cm,index=[clf.classes_],columns=[clf.classes_])
display(cm_df)
print()

accuracy = clf.score(X_test, y_test)
print('\nScore de Acuracidade:\n')
print(f'{accuracy:.4f}')

print('\nClassification Report:\n')
print(classification_report(y_test, y_pred))


KNeighborsClassifier()


Matriz de Confusão:

[[81  1]
 [ 1 47]]



,B,M
B,81,1
M,1,47




Score de Acuracidade:

0.9846

Classification Report:

              precision    recall  f1-score   support

           B       0.99      0.99      0.99        82
           M       0.98      0.98      0.98        48

    accuracy                           0.98       130
   macro avg       0.98      0.98      0.98       130
weighted avg       0.98      0.98      0.98       130



Q6. Qual o melhor k e a acuracidade do melhor modelo Knn?

3 e 0.9923

Q7. Para k=5, há quantos Falso Positivos e Falsos Negativos da classe B?

1 e 1

Q8. Para k=3, há quantos Falso Positivos e Falsos Negativos da classe B no melhor modelo Knn?

1 e 0

Q9. Para k=3, há quantos Falso Positivos e Falsos Negativos da classe M no melhor modelo Knn?

0 e 1

Q10. Podemos concluir desses testes que modelos K-vizinhos mais Próximos são melhores que modelos de Regressão Logística.

FALSO

Q11. Podemos concluir desses testes que quanto maior o K, melhores os modelos de K-vizinhos mais Próximos.

FALSO

Q12. Quantos casos benignos e malignos classificados nos novos casos com o melhor modelo de Knn?

67 e 41




In [15]:
# Novos casos
X_new = new_breast.drop(columns=['id','diagnosis'])
X_new = X_new / breast.drop(columns=['id','diagnosis']).max()
y_pred = clf.predict(X_new)

new_breast.diagnosis = y_pred

new_breast.diagnosis.value_counts()

B    68
M    40
Name: diagnosis, dtype: int64